# Eye movement panel of the poster

Looks only at right eye cameras

# Get data

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
# imports
import matplotlib as mpl
import seaborn as sns
from pathlib import Path
import pandas as pd
import warnings
import numpy as np
import matplotlib.pyplot as plt
import cv2

# from skimage.measure import EllipseModel
from mpl_toolkits.axes_grid1 import make_axes_locatable

import flexiznam as flz
from v1_depth_analysis.config import PROJECT
import v1_depth_analysis as vda
from cottage_analysis.eye_tracking import analysis as analeyesis
from cottage_analysis.eye_tracking import eye_model_fitting as emf

mpl.rcParams["pdf.fonttype"] = 42  # save text as text not outlines
from matplotlib.backends.backend_pdf import PdfPages

MOTION_CUTOFF = 0  # in degrees
VERSION = 1

save_root = Path(
    f"/camp/lab/znamenskiyp/home/shared/presentations/Cosyne2023/ver{VERSION}/"
)

/camp/home/hey2/.conda/envs/cottage_eye/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Get data for all sessions

In [7]:
# get session list
raw_path = Path(flz.PARAMETERS["data_root"]["raw"])
processed_path = Path(flz.PARAMETERS["data_root"]["processed"])
flm_sess = flz.get_flexilims_session(project_id=PROJECT)

recordings = vda.get_recordings(protocol="SpheresPermTubeReward", flm_sess=flm_sess)
datasets = vda.get_datasets(
    recordings, dataset_type="camera", dataset_name_contains="_eye", flm_sess=flm_sess
)
# keep only right eye cameras
datasets = [ds for ds in datasets if "right" in ds.dataset_name]
camera_datasets = {ds.full_name: ds for ds in datasets}

/camp/home/hey2/.conda/envs/cottage_eye/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: module 'pandas._libs.lib' has no attribute 'clean_index_list'

In [8]:
# get the data
data_by_recording = dict()
sampling_by_recording = dict()
no_behaviour = []
for cam_name, camera in camera_datasets.items():
    dlc_res, ellipse = analeyesis.get_data(
        camera,
        flexilims_session=flm_sess,
        likelihood_threshold=0.88,
        rsquare_threshold=0.99,
        error_threshold=3,
    )
    try:
        data, sampling = analeyesis.add_behaviour(
            camera,
            dlc_res,
            ellipse,
            speed_threshold=0.01,
            log_speeds=False,
            verbose=False,
        )
    except FileNotFoundError as err:
        warnings.warn(f"No data for {cam_name}")
        no_behaviour.append(cam_name)
    assert "valid" in data.columns
    # add trial number
    depth = np.array(data.depth.values, copy=True)
    depth[np.isnan(depth)] = -9999
    depth = np.round(depth, 2)
    trials_border = np.diff(np.hstack([-9999, depth]))
    trials_onset = np.where(trials_border > 5000)[0]
    trials_offset = np.where(trials_border < -5000)[0]
    trial_id = np.zeros(depth.shape) + np.nan
    for itrial, (on, off) in enumerate(zip(trials_onset, trials_offset)):
        trial_id[on:off] = itrial
    data["trial_id"] = trial_id
    data_by_recording[cam_name] = data
    sampling_by_recording[cam_name] = sampling
print(f"Loaded {len(data_by_recording)- len(no_behaviour)}/{len(datasets)} recordings ")

/camp/home/hey2/.conda/envs/cottage_eye/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: module 'pandas._libs.lib' has no attribute 'clean_index_list'

In [ ]:
# get the eye tracking fit
eye_parameters_by_recording = dict()
eye_rotations_by_recording = dict()
no_tracking = []
for cam_name, camera in camera_datasets.items():
    camera_save_folder = processed_path / camera.path / camera.dataset_name
    try:
        eye_parameters_by_recording[cam_name] = np.load(
            camera_save_folder / f"{camera.dataset_name}_eye_parameters.npz"
        )
    except FileNotFoundError as err:
        warnings.warn(f"No data for {cam_name}")
        no_tracking.append(cam_name)
        continue
    eye_rotations_by_recording[cam_name] = np.load(
        camera_save_folder / f"{camera.dataset_name}_eye_rotation_by_frame.npy"
    )
print(f"Loaded {len(eye_rotations_by_recording)}/{len(datasets)} recordings ")

In [ ]:
# Get camera extrinsics
calibration_folder = processed_path / PROJECT / "Calibrations"
calib_data = dict()
for cam_name in ["RightEyeCam", "LeftEyeCam"]:
    calib_data[cam_name.lower()] = dict()
    folder = calibration_folder / cam_name
    folder = list(folder.glob("*xtrinsics_flat"))[0]  # case is inconsistent
    folder = folder / "20220818" / "aruco5_5mm"
    assert folder.exists()
    for trial in folder.glob("trial*"):
        fname = str(trial / "camera_extrinsics_flat.yml")
        s = cv2.FileStorage()
        s.open(fname, cv2.FileStorage_READ)
        rvec = s.getNode("rvec").mat()
        tvec = s.getNode("tvec").mat()
        calib_data[cam_name.lower()][trial.name] = dict(rvec=rvec, tvec=tvec)
# take median across trials
extrinsics = dict()
for cam, trials in calib_data.items():
    extrinsics[cam] = dict()
    for w in ["rvec", "tvec"]:
        extrinsics[cam][w] = np.median(
            np.vstack([d[w].flatten() for d in trials.values()]), axis=0
        )

In [ ]:
# Get azimuth elevation
azel_by_recording = dict()
bad_number = []
valid_data = dict()
for cam_name, eye_rotation in eye_rotations_by_recording.items():
    if cam_name not in data_by_recording:
        print(f"Skipping bad session {cam_name}")
    camera = camera_datasets[cam_name]
    # get the camera we need for this acq and build tform matrix
    extrin = extrinsics[camera.dataset_name.replace("_", "")[:-3]]
    rmat, jac = cv2.Rodrigues(extrin["rvec"])
    gaze_vec = np.vstack([emf.get_gaze_vector(p[0], p[1]) for p in eye_rotation])
    world_gaze = emf.convert_to_world(gaze_vec, rmat=rmat)
    azimuth, elevation = emf.gaze_to_azel(world_gaze)

    # add that to dataframe
    # I need to cut the last few because of extra SI triggers

    azel_by_recording = np.vstack([azimuth, elevation])
    n = len(data_by_recording[cam_name])
    difference = len(azimuth) - n
    if (difference < 0) or (difference > 5):
        bad_number.append([cam_name, difference])
    else:
        valid_data[cam_name] = data_by_recording[cam_name].copy()
        valid_data[cam_name]["azimuth"] = np.rad2deg(
            azimuth[:n] - np.nanmedian(azimuth[:n])
        )
        valid_data[cam_name]["elevation"] = np.rad2deg(
            elevation[:n] - np.nanmedian(elevation[:n])
        )
print(f"Got {len(valid_data)} recordings at the end")
bad_number

In [ ]:
# add some useful measure
for cam_name, data in valid_data.items():
    data["delta_az"] = data.azimuth.diff()
    data["temporonasal_direction"] = np.sign(data.delta_az)
    data["delta_el"] = data.elevation.diff()
    data["nasal"] = np.nan
    data["temporal"] = np.nan
    data.loc[data.delta_az > 0, "nasal"] = np.abs(
        data.loc[data.delta_az > MOTION_CUTOFF, "delta_az"]
    )
    data.loc[data.delta_az < 0, "temporal"] = np.abs(
        data.loc[data.delta_az < -MOTION_CUTOFF, "delta_az"]
    )
    data["angle_of_motion"] = np.rad2deg(
        np.arctan2(np.deg2rad(data.delta_el), np.deg2rad(data.delta_az))
    )
    data["amplitude_of_motion"] = np.linalg.norm(
        np.vstack([data.delta_az, data.delta_el]), axis=0
    )
    data["angle_when_moving"] = np.nan
    data["saccade"] = data["amplitude_of_motion"] > (80 / sampling)
    data["saccade_direction"] = data.saccade * data.temporonasal_direction
    data["slow_motion"] = data.amplitude_of_motion.copy()
    data.loc[data.slow_motion > (80 / sampling), "slow_motion"] = np.nan
    moving = data.amplitude_of_motion > MOTION_CUTOFF
    data.loc[moving, "angle_when_moving"] = data.loc[moving, "amplitude_of_motion"]

In [ ]:
# remove session where the fit failed
badly_fitted = "PZAH6.4b_S20220519_R183410_SpheresPermTubeReward_right_eye_camera"
_ = valid_data.pop(badly_fitted)

In [ ]:
# aggregate data by session

by_session = []
av_trials = False
for cam_name, data in valid_data.items():
    in_corridor = data[~np.isnan(data.depth)].copy()
    by_trial = in_corridor.groupby(["trial_id"])
    m_by_trial = by_trial.aggregate(np.nanmean)
    if av_trials:
        # now average trial averages
        m_by_depth = m_by_trial.groupby(["depth"]).aggregate(np.nanmean)
    else:
        m_by_depth = m_by_trial
    m_by_depth = m_by_depth.reset_index()
    m_by_depth["session"] = cam_name
    by_session.append(m_by_depth)

by_session = pd.concat(by_session, ignore_index=True)
by_session.shape

# add cumulative motion
cumul_session = []
for cam_name, data in valid_data.items():
    in_corridor = data[~np.isnan(data.depth)].copy()
    by_trial = in_corridor.groupby(["trial_id"])
    m_by_trial = by_trial.aggregate(np.nansum)
    if av_trials:
        # now average trial averages
        m_by_depth = m_by_trial.groupby(["depth"]).aggregate(np.nanmean)
    else:
        m_by_depth = m_by_trial
    m_by_depth = m_by_depth.reset_index()
    m_by_depth["session"] = cam_name
    cumul_session.append(m_by_depth)
by_session_cumul = pd.concat(cumul_session, ignore_index=True)

## Get example session

In [ ]:
# camera_full_name = "PZAH6.4b_S20220419_R145152_SpheresPermTubeReward_right_eye_camera"
start_frame = 22000

camera_full_name = "PZAH6.4b_S20220512_R190248_SpheresPermTubeReward_right_eye_camera"

camera = [ds for ds in datasets if ds.full_name == camera_full_name]
camera = camera[0]
print(f"Analysing {' from '.join(camera.genealogy[::-1])}")
data = valid_data[camera_full_name]
eye_parameters = eye_parameters_by_recording[camera_full_name]
eye_rotations = eye_rotations_by_recording[camera_full_name]

In [ ]:
# Get example frame
video_file = camera.path_full / camera.extra_attributes["video_file"]
dlc_ds_name = "_".join(
    list(camera.genealogy[:-1]) + ["dlc_tracking", camera.dataset_name, "data", "0"]
)
dlc_ds = flz.Dataset.from_flexilims(name=dlc_ds_name, flexilims_session=flm_sess)
cropping = dlc_ds.extra_attributes["cropping"]
cam_data = cv2.VideoCapture(str(video_file))
cam_data.set(cv2.CAP_PROP_POS_FRAMES, start_frame - 1)
ret, frame = cam_data.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
cam_data.release()
gray = gray[cropping[2] : cropping[3], cropping[0] : cropping[1]]

# Plot the panel

In [ ]:
## Example eye

## binned data
elli = pd.DataFrame(data[data.valid], copy=True)
count, bin_edges_x, bin_edges_y = np.histogram2d(
    elli.pupil_x, elli.pupil_y, bins=(25, 25)
)
elli["bin_id_x"] = bin_edges_x.searchsorted(elli.pupil_x.values)
elli["bin_id_y"] = bin_edges_y.searchsorted(elli.pupil_y.values)
binned_ellipses = elli.groupby(["bin_id_x", "bin_id_y"])
ns = binned_ellipses.valid.aggregate(len)
binned_ellipses = binned_ellipses.aggregate(np.nanmedian)
mat = np.zeros((26, 26, 2)) + np.nan
for i_pos, (pos, series) in enumerate(binned_ellipses.iterrows()):
    mat[pos[1], pos[0]] = series[["azimuth", "elevation"]].values

# get other things we want to plot
eye_centre = eye_parameters["eye_centre"]
f_z0 = eye_parameters["f_z0"]
ellipse_model = emf.reproj_ellipse(
    *eye_rotations[start_frame], eye_centre=eye_centre, f_z0=f_z0
)
ref = data.iloc[start_frame][["reflection_x", "reflection_y"]].values

In [ ]:
depth_list

In [ ]:
# get depth color
depth_list = np.unique(data.depth)
depth_list = depth_list[~np.isnan(depth_list)]
cmap = mpl.cm.cool.reversed()
line_colors = []
norm = mpl.colors.Normalize(vmin=np.log(min(depth_list)), vmax=np.log(max(depth_list)))
col_dict = dict()
for depth in depth_list:
    rgba_color = cmap(norm(np.log(depth)), bytes=True)
    rgba_color = tuple(it / 255 for it in rgba_color)
    line_colors.append(rgba_color)
    col_dict[depth] = rgba_color

In [ ]:
def plot_example_eye(ax, fig):
    v = binned_ellipses[["pupil_x", "pupil_y"]].values
    lims = np.vstack([np.nanmin(v, axis=0), np.nanmax(v, axis=0)]) + ref
    circ_coord = ellipse_model.predict_xy(np.arange(0, 2 * np.pi, 0.1)) + ref.reshape(
        1, 2
    )
    vmin, vmax = np.quantile(gray, [0.01, 0.8])
    ax.imshow(gray, cmap="gray", vmin=vmin, vmax=vmax, zorder=-1)
    img = ax.imshow(
        mat[..., 0],
        extent=np.hstack([lims[:, 0], lims[::-1, 1]]),
        cmap="RdBu_r",
        vmin=-20,
        vmax=20,
        zorder=10,
    )
    ax.plot(
        circ_coord[:, 0],
        circ_coord[:, 1],
        label="Reprojection",
        color="lightblue",
        alpha=0.5,
        zorder=5,
    )
    pupil_c = np.array(ellipse_model.params[:2])
    ax.plot(*(eye_centre + ref), marker="o", ms=5, mfc="k", mec="none", zorder=1)
    ax.plot(
        *(pupil_c + ref), marker="o", ms=5, mfc="none", color="lightblue", alpha=0.5
    )
    ax.plot(
        *[(np.array([eye_centre[i], pupil_c[i]]) + ref[i]) for i in range(2)],
        color="lightblue",
        zorder=2,
        alpha=0.5
    )
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cb = fig.colorbar(img, cax=cax)
    cb.set_label("Azimuth (degrees)")
    ax.set_xlim([gray.shape[1], 0])
    ax.set_ylim([gray.shape[0] - 80, 50])
    ax.text(x=0.5, y=0.8, s="Dorsal", color="white", transform=ax.transAxes)
    ax.text(x=0.6, y=0.1, s="Nasal", color="white", transform=ax.transAxes)
    ax.axis("off")

In [ ]:
# time course
colors = [np.array([141, 160, 203]) / 255, np.array([252, 141, 98]) / 255]


def plot_time_course(
    ax,
    n_samples=int(120 * sampling),
    begin=20000,
    colors=colors,
    add_velocity=True,
    vel_by_depth=True,
):

    ax.axhline(0, color="grey", alpha=0.5, lw=1)
    time = np.arange(n_samples) / sampling

    azim = data.azimuth
    elev = data.elevation
    ax.plot(
        time, elev[begin : begin + n_samples], color=colors[1], label="Elevation", lw=1
    )
    ax.plot(
        time,
        azim[begin : begin + n_samples],
        color=colors[0],
        label="Azimuth",
        lw=1,
    )
    ax.set_xlabel("Time (s)")
    ax.set_ylabel(r"$\Delta$angle (degrees)")
    ax.set_xlim(0, time.max())
    ax.set_ylim(-20, 20)
    ax.set_xticks(np.arange(0, time.max(), 60))
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.legend(loc="lower right")
    divider = make_axes_locatable(ax)
    ax_h = divider.append_axes("right", size="10%", pad=0.05)
    bins = np.arange(-25, 25)
    ax_h.spines["right"].set_visible(False)
    ax_h.spines["top"].set_visible(False)
    ax_h.hist(
        elev,
        orientation="horizontal",
        density=True,
        bins=bins,
        color=colors[1],
        histtype="step",
        lw=2,
    )
    ax_h.hist(
        azim,
        orientation="horizontal",
        density=True,
        bins=bins,
        color=colors[0],
        histtype="step",
        lw=2,
    )
    ax_h.set_xlabel("Density")
    ax_h.set_yticks([])
    if add_velocity:
        ax_v = divider.append_axes("top", size="50%", pad=0.1)
        # ax_hv = divider.append_axes("top right", size="10%", pad=0.05, sharey=ax_v)
        # ax_hv.spines["right"].set_visible(False)
        # ax_hv.spines["top"].set_visible(False)
        bins = np.arange(0, 200)

        if vel_by_depth:
            depth = np.array(data.depth[begin : begin + n_samples])
            depth[np.isnan(depth)] = -1
            for d in np.unique(depth):
                mask = depth == d
                data_trace = np.array(
                    data.amplitude_of_motion[begin : begin + n_samples]
                )
                data_trace[~mask] += np.nan
                if d == -1:
                    color = np.array([0.7, 0.7, 0.7, 1])
                else:
                    color = line_colors[list(depth_list).index(d)]
                ax_v.plot(
                    time, data_trace * sampling, color=color, label="Velocity", lw=1
                )
        else:
            ax_v.plot(
                time,
                data.amplitude_of_motion[begin : begin + n_samples] * sampling,
                color=[0.2, 0.2, 0.2],
                label="Velocity",
                lw=1,
            )
        ax_v.set_ylim(0, 200)
        ax_v.set_ylabel(r"Velocity ($^{\circ}.s^{-1}$)")
        ax_v.set_xlim(ax.get_xlim())
        ax_v.set_xticks(np.arange(0, time.max(), 60))
        ax_v.set_xticklabels([])
    datapart = data[begin : begin + n_samples]
    for trial in datapart.trial_id.unique():
        if np.isnan(trial):
            continue
        b, e = datapart[datapart.trial_id == trial].index[[0, -1]]
        color = line_colors[list(depth_list).index(data.iloc[b].depth)]
        if not vel_by_depth:
            for x in ax, ax_v:
                x.axvspan(
                    *((np.array([b, e]) - begin) / sampling),
                    color=color,
                    zorder=-10,
                    alpha=0.5,
                    edgecolor="None"
                )
        else:
            for p in [b]:
                ax.axvline(
                    (p - begin) / sampling, color=color, zorder=-10, alpha=0.5, lw=1
                )


fig = plt.figure(figsize=(7, 3))
ax = plt.subplot(1, 1, 1)
plot_time_course(
    ax,
    n_samples=int(10 * 60 * sampling),
    begin=65000,
    add_velocity=True,
    colors=np.array([[0.2, 0.2, 0.2], [0.4, 0.4, 0.4]]),
)

In [ ]:
datapart = data[1000:10000]
for trial in datapart.trial_id.unique():
    if np.isnan(trial):
        continue
    b, e = datapart[datapart.trial_id == trial].index[[0, -1]]
b

In [ ]:
from matplotlib.collections import PolyCollection
from matplotlib.legend_handler import HandlerTuple


def label_violins(ax, title=None, hue_order=["nasal", "temporal"]):
    idepth = 0
    handles = []
    for ind, violin in enumerate(ax.findobj(PolyCollection)):
        color = line_colors[idepth]
        if ind % 2 != 0:
            color = 0.5 + 0.5 * np.array(color)  # make whiter
            idepth += 1
        violin.set_facecolor(color)
        handles.append(plt.Rectangle((0, 0), 0, 0, facecolor=color, edgecolor="black"))
    ax.legend(
        handles=[tuple(handles[::2]), tuple(handles[1::2])],
        labels=hue_order,
        title=title,
        handlelength=4,
        handler_map={tuple: HandlerTuple(ndivide=None, pad=0)},
    )

In [ ]:
fused_dataframe = []
for cam, cam_data in valid_data.items():
    cam_data["recording"] = cam
    cam_data["mouse"] = cam.split("_")[0]
    fused_dataframe.append(cam_data)
fused_dataframe = pd.concat(fused_dataframe, ignore_index=True)
for col in fused_dataframe.columns:
    try:
        fused_dataframe[col] = fused_dataframe[col].astype(float)
    except ValueError:
        print(f"Not changing {col}")

In [ ]:
rs_bins

In [ ]:
save_root

In [ ]:
plt.subplot(1, 2, 1)
plt.axvline(1, color="k")
_ = plt.hist(fused_dataframe.running_speed, bins=np.arange(0, 200))

np.save(save_root / "all_running_speeds.npy", fused_dataframe.running_speed.values)
plt.subplot(1, 2, 2)
_ = plt.hist(
    np.log10(fused_dataframe.running_speed[fused_dataframe.running_speed > 1].values),
    bins=np.arange(0, 3, 0.01),
)

In [ ]:
fig = plt.figure(figsize=(17.7165, 7.87402), constrained_layout=False)
# Eye tracking video example
ax = plt.subplot2grid((4, 12), (0, 0), rowspan=2, colspan=2)
plot_example_eye(ax, fig)

ax_timecourse = plt.subplot2grid((4, 12), (0, 3), rowspan=2, colspan=4)
plot_time_course(
    ax_timecourse,
    n_samples=int(10 * 60 * sampling),
    begin=65000,
    add_velocity=True,
    colors=np.array([[0.2, 0.2, 0.2], [0.4, 0.4, 0.4]]),
)
ax_timecourse.set_ylim([-21, 21])
# plot of position by depth
ax_medposition = plt.subplot2grid((4, 12), (0, 7), rowspan=2, colspan=2)

in_corridor = data[~np.isnan(data.depth)]
by_trial = in_corridor.groupby(["trial_id"])
m_by_trial = by_trial.aggregate(np.nanmean)
by_depth = m_by_trial.reset_index().groupby("depth")

m_by_depth = by_depth.aggregate(np.nanmean)
std_by_depth = by_depth.aggregate(np.nanstd)
for idepth, depth in enumerate(depth_list):
    ax_medposition.errorbar(
        m_by_depth.loc[depth].azimuth,
        m_by_depth.loc[depth].elevation,
        xerr=std_by_depth.loc[depth].azimuth,
        yerr=std_by_depth.loc[depth].elevation,
        marker="o",
        color=line_colors[idepth],
        label=int(depth),
    )
# ax_medposition.legend(loc="upper right", ncol=2)
ax_medposition.set_xlabel(r"Azimuth ($\circ$)")
ax_medposition.set_xlim([-5, 5])
ax_medposition.set_ylim([-5, 5])
ax_medposition.set_ylabel(r"Elevation ($\circ$)")


ax_sacc = plt.subplot2grid((4, 12), (0, 9), rowspan=2, colspan=2)
what2plot = "amplitude_of_motion"
factor = sampling
bins = np.linspace(0, np.nanmax(data[what2plot] * factor), 30)
obj = sns.histplot(
    ax=ax_sacc,
    x=data[what2plot] * factor,
    hue=data.depth,
    palette=line_colors,
    bins=bins,
    element="step",
    fill=False,
)
ax_sacc.semilogy()
ax_sacc.set_xlim([0, bins.max()])
ax_sacc.set_xlabel(r"Eye velocity ($^\circ.s^{-1}$)")
sns.move_legend(obj, loc="upper left", bbox_to_anchor=(1, 1))
if False:
    # give up on the polar plots
    ax_pos = [(0, 9), (0, 10), (1, 9), (1, 10), (1, 11)]
    width = 20
    bins = np.arange(-180, 180, width)

    for idepth, depth in enumerate(depth_list):
        ax_polar = plt.subplot2grid(
            (5, 12), ax_pos[idepth], rowspan=1, colspan=1, projection="polar"
        )
        d = data.loc[data.depth == depth]
        h, _ = np.histogram(
            d[d.amplitude_of_motion > (80 / sampling)].angle_of_motion, bins=bins
        )
        ax_polar.bar(
            np.deg2rad(bins[:-1]), h, width=np.deg2rad(width), color=line_colors[idepth]
        )
        ax_polar.set_rlim(0, 14)
        ax_polar.grid(False)
        ax_polar.set_xticks([])
        ax_polar.set_yticks([])

    ax_polar.set_xticks(np.arange(0, np.pi, np.pi / 2))
    ax_polar.set_xticklabels(["Nasal", "Dorsal"])
    ax_polar.set_yticks([14])

ax_all_sess = plt.subplot2grid((4, 12), (2, 0), rowspan=2, colspan=3)
azel_df = pd.DataFrame(
    dict(
        Axis=["Azimuth"] * len(by_session) + ["Elevation"] * len(by_session),
        Position=np.hstack([by_session.azimuth.values, by_session.elevation.values]),
        Depth=np.hstack([by_session.depth, by_session.depth]).astype(int),
    )
)
ax_all_sess.axhline(0, color="gray", alpha=0.5, zorder=-10)
sns.violinplot(
    azel_df,
    x="Depth",
    y="Position",
    hue="Axis",
    ax=ax_all_sess,
    split=True,
    hue_order=["Azimuth", "Elevation"],
)
label_violins(ax_all_sess, title=None, hue_order=["Azimuth", "Elevation"])
ax_all_sess.set_xlabel("Depth")
ax_all_sess.set_ylabel(r"Average eye position ($^\circ$)")

ax_all_sess = plt.subplot2grid((4, 12), (2, 3), rowspan=2, colspan=3)
if False:
    azel_df = pd.DataFrame(
        dict(
            Axis=["Nasal"] * len(by_session) + ["Temporal"] * len(by_session),
            Speed=np.hstack(
                [
                    by_session.nasal.values * sampling,
                    by_session.temporal.values * sampling,
                ]
            ),
            Depth=np.hstack([by_session.depth, by_session.depth]).astype(int),
        )
    )
    ax_all_sess.axhline(azel_df.Speed.median(), color="gray", alpha=0.5, zorder=-10)
    sns.violinplot(
        azel_df,
        x="Depth",
        y="Speed",
        hue="Axis",
        ax=ax_all_sess,
        hue_order=["Nasal", "Temporal"],
        split=True,
    )
    label_violins(ax_all_sess, title=None, hue_order=["Nasal", "Temporal"])
gpby = fused_dataframe.groupby(
    ["recording", "trial_id", "depth", "temporonasal_direction"]
)
df = gpby.aggregate(np.nanmean).reset_index()
df = df[df.temporonasal_direction != 0]
df.slow_motion *= sampling
sns.violinplot(
    data=df,
    x="depth",
    y="slow_motion",
    palette=line_colors,
    ax=ax_all_sess,
    split=True,
    hue_order=[1, -1],
    hue="temporonasal_direction",
)

label_violins(ax_all_sess)
ax_all_sess.set_xlabel("Depth")
ax_all_sess.set_ylabel(r"Average eye velocity ($^\circ.s^{-1}$)")
ax_all_sess.set_ylim([0, 30])
ax_all_sess.axhline(np.nanmedian(df.slow_motion), color="gray", alpha=0.5, zorder=-10)

if False:
    ax_all_sess = plt.subplot2grid((4, 12), (2, 7), rowspan=2, colspan=3)
    fused_with_gray = fused_dataframe.copy()
    fused_with_gray.loc[np.isnan(fused_with_gray.depth), "depth"] = 10000
    fused_with_gray.loc[:, "trial_id"] = -1
    print(fused_with_gray.depth.unique())
    gpby = fused_with_gray.groupby(
        ["recording", "trial_id", "depth", "temporonasal_direction"]
    )
    gdf = gpby.aggregate(np.nansum)
    trial = fused_with_gray.groupby(["recording", "trial_id", "depth"])
    ns = trial.aggregate(len)
    for (r, t, d, td) in gdf.index:
        gdf.loc[(r, t, d, td), "saccade"] /= ns.loc[(r, t, d), "saccade"]

    df = gdf.reset_index()
    sns.violinplot(
        x=df.depth,
        hue=df.temporonasal_direction,
        y=df.saccade * sampling,
        palette=list(line_colors) + [np.array([0, 0, 0, 1])],
        split=True,
        ax=ax_all_sess,
        hue_order=[1, -1],
    )

    label_violins(ax_all_sess)
    ax_all_sess.set_ylim(ax_all_sess.get_ylim()[0], 0.2)
    ax_all_sess.set_ylabel("Saccade rate (Hz)")
    ax_all_sess.axhline(
        np.nanmedian(df.saccade * sampling), color="gray", alpha=0.5, zorder=-10
    )


fig.subplots_adjust(hspace=1, wspace=1)

for ax in fig.axes:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

plt.savefig(save_root / "eye_tracking_panel.pdf")

In [ ]:
line_colors

In [ ]:
ax_all_sess = plt.subplot(1, 1, 1)
azel_df = pd.DataFrame(
    dict(
        Axis=["Nasal"] * len(by_session) + ["Temporal"] * len(by_session),
        Speed=np.hstack(
            [by_session.nasal.values * sampling, by_session.temporal.values * sampling]
        ),
        Depth=np.hstack([by_session.depth, by_session.depth]).astype(int),
    )
)
ax_all_sess.axhline(azel_df.Speed.median(), color="gray", alpha=0.5, zorder=-10)
sns.violinplot(
    azel_df,
    x="Depth",
    y="Speed",
    hue="Axis",
    ax=ax_all_sess,
    hue_order=["Nasal", "Temporal"],
    split=True,
)
label_violins(ax_all_sess, title=None, hue_order=["Nasal", "Temporal"])
ax_all_sess.set_xlabel("Depth")
ax_all_sess.set_ylabel(r"Average speed ($^\circ.s^{-1}$)")
ax_all_sess.set_ylim([0, 50])

In [ ]:
save_root

In [ ]:
ax_pos = [(0, 9), (0, 10), (0, 11), (1, 9), (1, 10)]
width = 20
bins = np.arange(-180, 180, width)
fig = plt.figure(figsize=(10, 5))

for imouse, mouse in enumerate(["PZAH", "PZAG"]):
    for idepth, depth in enumerate(depth_list):
        ax_polar = fig.add_subplot(2, 5, idepth + 1 + imouse * 5, projection="polar")
        if idepth == 0:
            ax_polar.set_ylabel(mouse)
        h = np.zeros(len(bins) - 1)
        for c, dat in valid_data.items():
            if not c.startswith(mouse):
                continue
            d = dat.loc[dat.depth == depth]
            hd, _ = np.histogram(
                d[d.amplitude_of_motion > (80 / sampling)].angle_of_motion, bins=bins
            )
            h += hd
        ax_polar.bar(
            np.deg2rad(bins[:-1]), h, width=np.deg2rad(width), color=line_colors[idepth]
        )
        # ax_polar.set_rlim(0, 14)
        ax_polar.set_xticks(np.arange(0, np.pi, np.pi / 2))
        ax_polar.set_xticklabels(["Nasal", "Dorsal"])
        # ax_polar.set_yticks([14])
        ax_polar.grid(False)

In [ ]:
fused_dataframe.recording.dtype

In [ ]:
gpby = fused_dataframe.groupby(
    ["recording", "trial_id", "depth", "temporonasal_direction"]
)
df = gpby.aggregate(np.nanmean).reset_index()
df = df[df.temporonasal_direction != 0]
df.slow_motion *= sampling
ax = plt.subplot(1, 1, 1)
sns.violinplot(
    data=df,
    x="depth",
    y="slow_motion",
    palette=line_colors,
    ax=ax,
    split=True,
    hue_order=[1, -1],
    hue="temporonasal_direction",
)

label_violins(ax)
av = df.groupby(["depth", "temporonasal_direction"]).aggregate(np.nanmean).slow_motion

In [ ]:
a = av.reset_index()
for d, ad in a.groupby("depth"):
    c = line_colors[list(depth_list).index(d)]
    plt.plot(ad.temporonasal_direction, ad.slow_motion, marker="o", color=c)

In [ ]:
list(line_colors)

In [ ]:
gpby = fused_dataframe.groupby(
    ["recording", "trial_id", "depth", "temporonasal_direction"]
)
gdf = gpby.aggregate(np.nansum)
trial = fused_dataframe.groupby(["recording", "trial_id", "depth"])
ns = trial.aggregate(len)

for (r, t, d, td) in gdf.index:
    gdf.loc[(r, t, d, td), "saccade"] /= ns.loc[(r, t, d), "saccade"]

df = gdf.reset_index()
ax = plt.subplot(1, 1, 1)
sns.violinplot(
    x=df.depth,
    hue=df.temporonasal_direction,
    y=df.saccade * sampling * 60,
    palette=line_colors,
    split=True,
    ax=ax,
    hue_order=[1, -1],
)

label_violins(ax)
ax.set_ylabel("Saccade/minute")
# ax.set_ylim(-1, 5)

In [ ]:
trial

In [ ]:
recs = fused_dataframe.recording.unique()

rec = recs[3]
rec_df = fused_dataframe[fused_dataframe.recording == rec]
rng = np.random.default_rng(seed=9876)
for idepth, depth in enumerate(depth_list):
    ddf = rec_df[rec_df.depth == depth]
    trials = ddf.trial_id.unique()
    rand = rng.integers(0, len(trials))
    trial = trials[rand]
    med = 0  # np.nanmedian(ddf.amplitude_of_motion)
    for trial in trials:
        trial_exp = ddf[ddf.trial_id == trial]
        time = np.arange(len(trial_exp.amplitude_of_motion)) / sampling
        plt.plot(
            time,
            trial_exp.amplitude_of_motion + idepth * 10 - med,
            color=line_colors[idepth],
        )
plt.xlim(0, 60)

In [ ]:
rec_df.trial_id.unique()

In [ ]:
trials_border

In [ ]:
trials_border

In [ ]:
recs = fused_dataframe.recording.unique()
rec = recs[3]
rec_df = fused_dataframe[fused_dataframe.recording == rec].copy()

rec_df.loc[np.isnan(rec_df.trial_id), "trial_id"] = -1
n_trials = 30
first_trial = 51


trials_border = [
    np.where(rec_df.trial_id == t)[0][[0, -1]]
    for t in np.arange(n_trials) + first_trial
]
n_samples = 0
b, e = trials_border[0][0], trials_border[-1][0]
t0 = b / sampling
non_corr = rec_df.iloc[b:e].trial_id != -1
time = np.arange(b, e) / sampling - t0
bg = np.array(rec_df.iloc[b:e].amplitude_of_motion.copy()) * sampling
bg[non_corr] += np.nan
fig = plt.figure(figsize=(7, 2))
plt.plot(time, bg, color="gray", lw=1)

for i, (b, e) in enumerate(trials_border):
    time = np.arange(b, e) / sampling - t0
    depth = rec_df.iloc[b + 1].depth
    color = line_colors[list(depth_list).index(depth)]
    plt.plot(
        time, rec_df.iloc[b:e]["amplitude_of_motion"] * sampling, color=color, lw=1
    )
plt.xlabel("Time (s)")
plt.ylim(0, 200)
plt.xlim(0, time[-1])

In [ ]:
ddf = fused_dataframe[fused_dataframe.depth == depth]
recs = ddf.groupby(["recording", "trial_id"])
keys = recs.groups.keys()
rand = rng.integers(0, len(keys))
gpd = recs[list(keys)[rand]]

In [ ]:
l = np.array(
    [
        ns.loc[(r, t, d), "saccade"]
        for (r, t, d, di) in gdf[gdf.saccade > 5 / (sampling * 60)].index
    ]
)
plt.hist(ns.saccade.values / sampling, bins=np.arange(60 * 2))
plt.hist(l / sampling, bins=np.arange(60 * 2))

In [ ]:
gdf[gdf.saccade > 5 / (sampling * 60)]

In [ ]:
idx = pd.IndexSlice

df.loc[idx[r, t, :, :], "saccade"]

In [ ]:
ns = (
    fused_dataframe.groupby(["recording", "trial_id", "depth"])
    .aggregate(len)
    .reset_index()
)
ns["saccade"] = ns["saccade"] / sampling
sns.violinplot(data=ns, x="depth", y="saccade", palette=line_colors)
plt.gca().set_ylabel("Trial duration (s)")
plt.gca().set_ylim(0, 50)
plt.axhline(ns["saccade"].median(), color="gray", alpha=0.5, zorder=-10)

In [ ]:
# create of and rs bins
log_range = {
    "rs_bin_log_min": 0,
    "rs_bin_log_max": 2.5,
    "rs_bin_num": 6,
    "of_bin_log_min": -1.5,
    "of_bin_log_max": 3.5,
    "of_bin_num": 11,
    "log_base": 10,
}

rs_bins = np.logspace(
    log_range["rs_bin_log_min"],
    log_range["rs_bin_log_max"],
    num=log_range["rs_bin_num"],
    base=log_range["log_base"],
)
of_bins = np.logspace(
    log_range["of_bin_log_min"],
    log_range["of_bin_log_max"],
    num=log_range["of_bin_num"],
    base=log_range["log_base"],
)
fused_dataframe["rs_bin"] = np.nan
fused_dataframe["of_bin"] = np.nan
mask = ~np.isnan(fused_dataframe.running_speed)
fused_dataframe.loc[mask, "rs_bin"] = rs_bins.searchsorted(
    fused_dataframe.loc[mask, "running_speed"]
)
mask = ~np.isnan(fused_dataframe.optic_flow)
fused_dataframe.loc[mask, "of_bin"] = of_bins.searchsorted(
    fused_dataframe.loc[mask, "optic_flow"]
)

In [ ]:
ddf = (
    fused_dataframe.groupby(["recording", "trial_id", "depth"])
    .aggregate(np.nanmean)
    .reset_index()
)

fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(4, 1, 1)
sns.violinplot(data=ddf, x="depth", y="running_speed", palette=line_colors, ax=ax)
ax = plt.subplot(4, 1, 2)
sns.violinplot(data=ddf, x="depth", y="rs_bin", palette=line_colors, ax=ax)
ax = plt.subplot(4, 1, 3)
sns.boxenplot(data=ddf, x="depth", y="saccade", palette=line_colors, ax=ax)
ax.set_ylim(0, 0.05)
ax = plt.subplot(4, 1, 4)
sns.violinplot(data=ddf, x="depth", y="amplitude_of_motion", palette=line_colors, ax=ax)

In [ ]:
fused_dataframe["depth"] = fused_dataframe["depth"].astype(float)

In [ ]:
ddf = (
    fused_dataframe.groupby(["recording", "trial_id", "of_bin"])
    .aggregate(np.nanmean)
    .reset_index()
)

fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(4, 1, 1)
sns.violinplot(data=ddf, x="of_bin", y="running_speed", palette=line_colors, ax=ax)
ax = plt.subplot(4, 1, 2)
sns.violinplot(data=ddf, x="of_bin", y="depth", palette=line_colors, ax=ax)
ax = plt.subplot(4, 1, 3)
sns.violinplot(data=ddf, x="of_bin", y="optic_flow", palette=line_colors, ax=ax)
ax = plt.subplot(4, 1, 4)
sns.violinplot(
    data=ddf, x="of_bin", y="amplitude_of_motion", palette=line_colors, ax=ax
)

In [ ]:
ddf = (
    fused_dataframe.groupby(["recording", "trial_id", "rs_bin"])
    .aggregate(np.nanmax)
    .reset_index()
)

fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(4, 1, 1)
sns.violinplot(data=ddf, x="rs_bin", y="running_speed", palette=line_colors, ax=ax)
ax = plt.subplot(4, 1, 2)
sns.violinplot(data=ddf, x="rs_bin", y="depth", palette=line_colors, ax=ax)
ax = plt.subplot(4, 1, 3)
sns.violinplot(data=ddf, x="rs_bin", y="optic_flow", palette=line_colors, ax=ax)
ax = plt.subplot(4, 1, 4)
sns.boxenplot(data=ddf, x="rs_bin", y="amplitude_of_motion", palette=line_colors, ax=ax)